# EVALUATION (OBTENTION OF METRICS) OF THE DIFFERENT MODELS USING SPACY-LLM


#DOING NER

Installing required libraries

In [ ]:
!pip install spacy-llm
!pip install spacy[transformers,lookups]
!pip install confection
!pip install openai
!pip install spacy[llm]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

In [ ]:
!pip install --upgrade spacy

Importing spacy and openai

In [ ]:
import spacy
import openai
from google.colab import userdata
import os
from spacy_llm.util import assemble
import json

Set the OpenAI API KEY

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('apiKey')

Function that gets the entities from a certain report, and places them in a dictionary

In [ ]:
def get_entities(nlp, report):
    doc = nlp(report)
    entities = []
    for ent in doc.ents:
        entity_info = {
            "label": ent.label_,
            "start": ent.start_char,  # Start position in the text
            "end": ent.end_char  # End position in the text
        }
        entities.append(entity_info)  # Append the entity information to the list
    return entities

Function that gets a dictionary and creates a json

In [ ]:
def create_json(entities, report_name):
  with open(f"{report_name}Entities4Metrics.json", "w") as outfile:
    json.dump(entities, outfile)

Function that processes a report

In [39]:
def process_report(report, report_name, nlp):
  entities = get_entities(nlp, report)
  return entities


#PREPARING THE NER-RESULTS FOR EVALUATION


Function that processes all of the reports from the input json file. It returns the list of entities recognised by the Model for each report. The list is provided in a format suitable for evaluation.

In [36]:
def process_input_json(file_name, nlp):
  dictionary = json.load(open(file_name))
  list_of_lists_of_entities = []
  for key, value in dictionary:
    list_of_lists_of_entities.append(process_report(value, key, nlp))
  return list_of_lists_of_entities

Function that creates the nlp:

In [ ]:
def create_nlp():
  nlp = assemble("config.cfg")
  return nlp

Function that gets a list of true entities present in the report and transforms it into suitable format.

In [ ]:
#Expected format: entities
'''
entities = [
    ("Names of Patients", "James Turner"),
    ("Patient's Age", "45"),
    ("Patient's Sex", "Male"),
    ("Patient's Height", "175 cm"),
    ("Patient's Weight", "88 kg"),
    ("Admission date", "13th August 2023"),
    ("Medical discharge date", "20th August 2023"),
    ("Diagnosis", "chronic obstructive pulmonary disease (COPD)"),
    ("Symptom", "persistent cough"),
    ("Symptom", "shortness of breath"),
    ("Medication and dosing", "Tiotropium, 18 mcg once daily"),
    ("Medication and dosing", "Salbutamol inhaler, 100 mcg as needed"),
    ("Tests", "pulmonary function test")
]
text = "**Clinical Report 1**  \nPatient Name: James Turner  \nAge: 45  \nSex: Male  \nHeight: 175 cm  \nWeight: 88 kg  \n\nDuring Mr. Turner's recent examination on 14th August 2023, he presented with persistent cough and shortness of breath. After conducting a pulmonary function test, he was diagnosed with chronic obstructive pulmonary disease (COPD). Treatment included the administration of Tiotropium, 18 mcg once daily, and Salbutamol inhaler, 100 mcg as needed. He was admitted on 13th August 2023 and discharged on 20th August 2023. Follow-up is scheduled in four weeks to assess response to treatment.  \n"
'''

In [37]:
def parse(text, entities):
  pos_entities = 0
  lista = []
  i = 0
  while i < len(text) and pos_entities < len(entities):
    nomEnt,string = entities[pos_entities]
    if(i+len(string) < len(text) and  text[i:i+len(string)] == string):
      lista.append({"start":i,"end":i+len(string),"label":nomEnt})
      pos_entities += 1
    i += 1
  return lista

Función que procesa el fichero json con las etiquetas verdaderas de los textos:

In [38]:
def process_input_true_labels_json(file_name):
  dictionary = json.load(open(file_name))
  list_of_lists_of_entities = []
  for key, value in dictionary:
    list_of_lists_of_entities.append((key,value))
  return list_of_lists_of_entities

Main function:

In [30]:
def main():
  nlp = create_nlp()
  predicted_labels = process_input_json("medical_reports.json", nlp)
  print(predicted_labels)
  ground_truth_labels = process_input_true_labels_json("true_labels.json")
  print(ground_truth_labels)
  #get_metrics(predicted_labels, ground_truth_labels, nlp)


Program:

In [40]:
main()

ValueError: too many values to unpack (expected 2)

#Obtención Automatizada de Métricas


In [ ]:
!pip install nervaluate

In [ ]:
from nervaluate import Evaluator
def get_metrics(predicted_labels, ground_truth_labels, nlp):
  evaluator = Evaluator(ground_truth_labels, predicted_labels, tags = list(nlp.get_pipe("llm").labels) , loader="default")

  # call the evaluator to get model evaluation result
  results, results_by_tag = evaluator.evaluate()
  print(results)
  print(results_by_tag)